In [ ]:
import requests
import json
import math
import datetime
import time
import os
import pandas as pd


##### How to use: in the command line type coinprice(symbol,start_date,day_range) , but with your values for each arguement
    #3 see below for what the arguments are

def coinprice(symbol,start_date,day_range): #symbol is coin ticker, all caps entered between quotes: Example -'ADA'
                                            #start date format is month/day/year in quaotes: Example -'03/20/2019' 
                                        #day range,I recommended use only multiples of 10 days: Example 180 days is 6 months
            
    t0 = int(pd.to_datetime(start_date).timestamp()*1e3) ## convert date and time to unix for... 
                                                        # ...data start range, way easier to work with

    coin_price = [] # allocate space for empty list for coin prices to populate 
    dates = [] # allocate space for empty list for dates to populate 
    
    for i in range(int(day_range/10)): #for loop gets data in 10 day chunks, hence divide day range by 10
        
        para = {'symbol':symbol+'USDT', #create api payload with coin name/usd pairing 
                'interval':'15m',    # use 15 minute price intervals
                'startTime':int(t0), # start time
                'endTime':int(t0 + 8640000*1e3), # endtime
                'limit':960}  # number of data points (960 fifteen minute chunks equals 10 full days)

        price_info = requests.get('https://api.binance.com/api/v3/klines',params=para).json() # make the api call for data
        t0 +=864000*1e3 # add 10 days each iteration of i
        
        for j in range(960): # this for loop populates empty list with coin price and  corresponding dates
            coin_price.append(price_info[j][3])
            dates.append(price_info[j][0])    




    df = pd.DataFrame(list(zip(dates, coin_price)), #combine all data into dataframe
                   columns =['date', 'price'])
    
    df['price'] = pd.to_numeric(df['price'], downcast="float") #convert prices to numeric values
    df['date'] = pd.to_datetime(df['date'],unit='ms') #convert date to readable date
    df.date = df["date"].dt.strftime("%m/%d/%y") #pick date format
    
    
    cwd = os.getcwd()
    name = symbol+ 'PriceInfo'
    df.to_csv(name + '.csv', sep=',',index=False)
    
    print('your price data can be found in ' + str(cwd)+', its named '+name+'.csv')
    return df
